In [2]:
from huggingface_hub.utils import insecure_hashlib

In [1]:
import pandas as pd
import os
from datasets import load_dataset

In [2]:
data = pd.read_csv("Fake.csv")

In [3]:
our_data = load_dataset('GonzaloA/fake_news')

Using the latest cached version of the dataset since GonzaloA/fake_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\KIRKE\.cache\huggingface\datasets\GonzaloA___fake_news\default\0.0.0\d653ddbf8eecee268bf6bc6e2fb2d0433704fedf (last modified on Mon Apr 29 14:35:09 2024).


In [4]:
data.title

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
23476    McPain: John McCain Furious That Iran Treated ...
23477    JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478    Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479    How to Blow $700 Million: Al Jazeera America F...
23480    10 U.S. Navy Sailors Held by Iranian Military ...
Name: title, Length: 23481, dtype: object

In [5]:
sampled = data.title

In [6]:
from collections import defaultdict
d = defaultdict(dict)
c = 0
for counter, text in enumerate(our_data["train"]["title"]):
    found = False
    for text_2 in sampled:
        if text in text_2:
            d[counter]["our_data"] = text
            d[counter]["sampled"] = text_2
            d[counter]["missing"] = False
            found = True
            break
    if not found:
        d[counter]["our_data"] = None
        d[counter]["sampled"] = text
        d[counter]["missing"] = True

In [7]:
miav = pd.DataFrame(d).T

{0,
 1,
 3,
 4,
 5,
 8,
 9,
 14,
 19,
 26,
 28,
 29,
 32,
 34,
 35,
 37,
 38,
 40,
 46,
 47,
 50,
 54,
 55,
 57,
 58,
 60,
 61,
 63,
 65,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 77,
 80,
 81,
 82,
 86,
 87,
 95,
 98,
 99,
 101,
 106,
 108,
 115,
 116,
 118,
 120,
 121,
 128,
 130,
 133,
 134,
 135,
 140,
 145,
 149,
 150,
 152,
 157,
 158,
 159,
 160,
 163,
 164,
 166,
 167,
 170,
 171,
 175,
 176,
 179,
 180,
 182,
 191,
 195,
 196,
 200,
 203,
 204,
 206,
 207,
 208,
 211,
 212,
 213,
 215,
 216,
 217,
 220,
 221,
 225,
 231,
 232,
 233,
 235,
 236,
 238,
 244,
 245,
 248,
 249,
 252,
 253,
 254,
 255,
 256,
 257,
 260,
 261,
 264,
 265,
 267,
 278,
 279,
 282,
 283,
 288,
 289,
 292,
 293,
 294,
 295,
 296,
 298,
 299,
 300,
 301,
 302,
 304,
 308,
 311,
 313,
 314,
 315,
 317,
 318,
 319,
 329,
 335,
 338,
 339,
 341,
 342,
 345,
 349,
 352,
 353,
 354,
 356,
 360,
 361,
 362,
 367,
 371,
 378,
 382,
 383,
 384,
 389,
 391,
 394,
 398,
 404,
 405,
 407,
 408,
 411,
 414,
 415,
 41

In [17]:
with open("true.txt","r") as f:
    x = f.read()

In [25]:
true = set([int(x) for x in x.split("\n")])

In [30]:
merged = true.union(set(miav[miav["missing"] == False].index))

In [34]:
li = []
for i in range(max(merged)):
    if i not in merged:
        li.append(i)
len(li)

1155

## The data

## Reflection on fake news

Fake news has many definitions, ranging from being factually incorrect to misleading, which makes it hard when quantifying results and cross examining results between different studies, since the latent space can be different dependent on the definition of the task. The problem with using objective truth as the definition, that truth can vary depending on culture and context. An actor in a conflict can be seen as the good freedome fighter towards an opressive regime by one side, and as a terrorist on the other side. Lastly, the truth can also change over time, which means that models have to be retrained with up to date information constantly to be able to combat the fake news within the catagory factual correct. Using the definition of misleading would be an inherently easier task since the model would not have to have a world view in order to classify. It should be solveable by using information only present in the text and comparing that to the title. 

## Related work

<!-- Recent studies have shown that fake news propagates through social media at unprecedented speeds. This was observed to happen during the emergence of COVID-19, thus the need to quickly detect and mitigate the spreading of fake news is more important than ever s[1]. 

Many definitions are presented, ranging from being factually incorrect to misleading, and unfortunately, our data source has not specified which definition they use. This makes it harder to interpret why a model predicted as it did, since we do not know if the data contains mostly stories conflicting with reality, or simply written by an overselling journalist. -->

Related work
This paper, written by Shaina Raza & Chen Ding, uses META's BART language model trained on two data sets: NELA-GT-19, which are news articles sourced from multiple sites, and Fakeddit, which is a multimodal dataset from Reddit, consisting of both images and text. The datasets used had more than a binary score, it included labels such as mixed, which is when there is a disagrement whether something is true or false, and categories such as satire into a single category Fake. They discuss their approach of continuously updating the model's training data to retrain the model and stay on top of relevant news. They also assert that freezing a model's weights can quickly make the model outdated since they don't generalize well to future events. Finally, they report an accuracy of 74.89%.



s[1] = https://link.springer.com/article/10.1007/s41060-021-00302-z

s[2] = https://arxiv.org/pdf/2101.00180.pdf